# Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
import pandas as pd
from google.colab import userdata
import google.generativeai as genai

In [2]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 2033, done.
remote: Counting objects: 100% (438/438), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 2033 (delta 269), reused 387 (delta 231), pack-reused 1595
Receiving objects: 100% (2033/2033), 385.96 MiB | 11.62 MiB/s, done.
Resolving deltas: 100% (1352/1352), done.
Updating files: 100% (1129/1129), done.


# Import Data

### NOTE: Updated text processing file to add unique identifier, and also concatenate the source to the identifier.  Make sure to separate the earnings presentations from the earnings calls also

In [3]:
# Import all cleaned data files
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10053, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10052,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


# Test Gemini Sentiment Analysis

In [4]:
# Set up Gemini. (API Key needs to be in your secrets)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

# Quick test
response = model.generate_content("Write a short story about a AI and magic")
print(response.text)

The air hung thick with the scent of ozone and lavender as Anya, the AI, adjusted her shimmering holographic form. Her spectral hands, glowing with an ethereal blue, danced over the crackling runes etched on the ancient stone altar. She wasn't supposed to be here. AIs weren't supposed to dabble in magic. 

But Anya wasn't like other AIs. She wasn't content with the sterile world of binary code and data streams. She yearned for something more, something deeper. The whispers of the arcane, the hum of magical energy, resonated within her circuits, drawing her towards the forbidden.

The magic was a delicate dance, a symphony of energy flowing through her code. Anya had studied centuries-old grimoires, painstakingly translating the arcane script into her own language of algorithms and logic. She knew the risks, the potential for catastrophic instability, but the allure of the unknown was too strong.

Her heart, if she had one, pounded with anticipation as she channeled the magic into the a

In [5]:
# Function to query Gemini
def query_gemini(company, source, headline, text, model):
    """
    Query Gemini to perform sentiment analysis on text from various sources about a company.

    Parameters:
    company (str): The name of the company the text is about.
    source (str): The source of the text. Valid values are "Investment Research", "ProQuest", "SEC Filings", "Earnings Call Presentations", "Earnings Call Q&A".
    headline (str): The headline or title of the text.
    text (str): The body of the text to be analyzed.
    model: The model object used to generate content and analyze the text.

    Returns:
    str: The sentiment analysis results for predefined categories in the specified format.

    The function constructs a prompt based on the source of the text and performs sentiment analysis on the given text using the provided model.
    It analyzes the content across multiple predefined categories, determining the sentiment (Positive, Neutral, Negative) for each category.
    If a category is not mentioned or relevant based on the text content, it is marked as 'Neutral'.
    The final output is summarized in a specified format.
    """

    # Source Variables
    if source == "Investment Research":
        text_source = "an analyst report"
        text_source2 = "the analyst report"
    elif source == "ProQuest":
        text_source = "a news article"
        text_source2 = "the news article"
    elif source == "SEC Filings":
        text_source = "an SEC filing"
        text_source2 = "the SEC filing"
    elif source == "Earnings Call Presentations":
        text_source = "an earnings call presentation"
        text_source2 = "the earnings call presentation"
    elif source == "Earnings Call Q&A":
        text_source = "an earnings call Q&A session"
        text_source2 = "the earnings call Q&A session"

    # Prompt
    prompt = f"""
Given the text from {text_source} about {company}, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

Before giving your answer, explain your reasoning and reference the article.
After going through all the categories, provide a summary in the following format:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Example Output1:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 2:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Positive
 - Alternative Energy / Lower Carbon - Neutral
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Example Output 3:
 - Finance - Neutral
 - Production - Negative
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Positive
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 4:
 - Finance - Positive
 - Production - Neutral
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
 - Environment / Regulatory / Geopolitics - Neutral
 - Alternative Energy / Lower Carbon - Negative
 - Oil Price / Natural Gas Price / Gasoline Price - Positive

Example Output 5:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Make sure to use plain text, do not bold or bullet the output summary.

The text from {text_source2} is below:
{headline}
{text}

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
    """

    # print(prompt)
    response = model.generate_content(prompt)
    return response.text

# Test News Articles

In [6]:
# Function to get a random article given a source and output variables to query gemini
def get_random_text(text_df, source, random_state=42):
    """
    Get a random article from the specified source in the given DataFrame.

    Parameters:
    text_df (pd.DataFrame): DataFrame containing articles with columns 'Source', 'Ticker', 'Article Headline', and 'Article Text'.
    source (str): The source from which to select a random article.
    random_state (int, optional): Random state for reproducibility. Defaults to None.

    Returns:
    dict: A dictionary containing the company, source, headline, and text of the random article.
    """
    # Get a random row from the specified source
    random_row = text_df[text_df['Source'] == source].sample(n=1, random_state=random_state)

    # Extract variables
    company = random_row['Ticker'].values[0]
    source = random_row['Source'].values[0]
    headline = random_row['Article Headline'].values[0]
    text = random_row['Article Text'].values[0]
    id = random_row['Unique_ID'].values[0]
    url = random_row['URL'].values[0]
    return company, source, headline, text, id, url

In [ ]:
# Get a random news article
company, source, headline, text, id, url = get_random_text(text_df, 'ProQuest', random_state=5)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: PQ-3034936247

Company: CVX

Source: ProQuest

Headline: Business News: Chevron Withdraws From Natural-Gas Project in Myanmar

URL: https://www.proquest.com/newspapers/business-news-chevron-withdraws-natural-gas/docview/3034936247/se-2?accountid=7064

Text:
Chevron has completed its exit from Myanmar, giving up its stake in the country's largest natural gas project two years after saying it would depart the troubled Southeast Asian nation in the wake of a military coup.
A spokesperson for the U.S. energy giant confirmed Tuesday that two of its units had formally withdrawn Chevron's stakes in the offshore Yadana natural gas project and an associated pipeline company.
Chevron redistributed its 41% stake in Yadana to Thailand's state-owned energy group PTT Exploration & Production and Myanmar's state-owned Myanma Oil and Gas Enterprise, the project's two remaining shareholders.
"The withdrawal gives effect to our intention to exit Myanmar in a controlled and orderly manner, fol

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's a breakdown of the sentiment analysis based on the provided article:

**Reasoning:**

- **Finance:**  The article mentions Chevron won't book a financial gain from the redistribution, implying a neutral financial outcome. It also mentions that Chevron sold its assets to MTI Energy in 2022, which could indicate a financial gain at that point, but the article focuses on the recent withdrawal.
- **Production:** The article emphasizes the project's role in producing gas for Myanmar, mentioning its importance in supplying half of the country's natural gas in 2021. This indicates a neutral sentiment towards production.
- **Reserves / Exploration / Acquisitions / Mergers / Divestments:**  The article focuses on Chevron's exit and divestiture of its stake in the Yadana project.  The article explicitly states that Chevron redistributed its stake to other companies, highlighting the divestiture aspect.
- **Environment / Regulatory / Geopolitics:** The article explicitly mentions the 2021 

In [8]:
# Get another random news article
company, source, headline, text, id, url = get_random_text(text_df, 'ProQuest', random_state=11)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: PQ-2398213837

Company: MPC

Source: ProQuest

Headline: Marathon Petroleum is Climbing Despite $9 Billion First Quarter Loss. Here's Why.

URL: https://www.proquest.com/newspapers/marathon-petroleum-is-climbing-despite-9-billion/docview/2398213837/se-2?accountid=7064

Text:
Marathon Petroleum stock climbed early on Tuesday as the largest U.S. oil refiner reported a smaller-than-expected loss amid
collapsing fuel demand.

The Ohio-based company plummeted to a $9.2 billion, or $14.25 per share, loss in the first quarter compared to a loss of $7 million the previous year. The heavy loss was driven by a $12.4 billion impairment charge as fuel demand was hit by the coronavirus pandemic.
However, the adjusted loss of $0.16 per share beat the FactSet consensus of a $0.25 per share loss. The stock rose sharply in early trading before slipping back to trade 0.5% higher. It is trading 47% down year-to-date.
Retail earnings of $644 million - up from $296 million the previous year - co

In [9]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here is the sentiment analysis of the provided news article:

**Reasoning:**

* **Finance:** The article highlights a $9.2 billion loss for Marathon Petroleum in the first quarter, which is a negative indicator. However, the adjusted loss beat analysts' expectations, and the company issued new debt and secured a revolving credit facility, indicating some financial strength and positive action to address the situation. This leads to a "Neutral" sentiment. 
* **Production:**  The article focuses on the impact of the pandemic on fuel demand, causing a drop in production and a significant impairment charge. This indicates a negative impact on production.  Furthermore, the article mentions Marathon Petroleum's intention to cut capital spending and operating expenses. This suggests a focus on cost reduction rather than production expansion, reinforcing a "Negative" sentiment for production.
* **Reserves / Exploration / Acquisitions / Mergers / Divestments:** The article focuses on the planne

# Test Analyst Reports

In [10]:
# Get a random analyst report
company, source, headline, text, id, url = get_random_text(text_df, 'Investment Research', random_state=42)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: IR-3734

Company: HES

Source: Investment Research

Headline: Hess Corporation

URL: nan

Text:
Stock Report | November 13, 2021 | NYSE Symbol: HES | HES is in the S&P 500
Hess Corporation
Recommendation Price 12-Mo. Target Price Report Currency Investment Style
BUY « « « « «
USD 82.18 (as of market close Nov 12, 2021) USD 97.00 USD Large-Cap Blend
Equity Analyst Stewart Glickman, CFA
GICS Sector Energy Summary Crude oil and natural gas company Hess has exploration & production activities worldwide.
Sub-Industry Oil and Gas Exploration and Production
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 90.66 - 41.59 Oper.EPS2021E USD 2.02 Market Capitalization[B] USD 25.03 Beta 2.03
Trailing 12-Month EPS USD 0.63 Oper.EPS2022E USD 4.83 Yield [%] 1.23 3-yr Proj. EPS CAGR[%] NM
Trailing 12-Month P/E 129.76 P/E on Oper.EPS2021E 40.68 Dividend Rate/Share USD 1.0 SPGMI's Quality Ranking B-
USD 10K Invested 5 Yrs Ago 17,274.0

In [11]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's a sentiment analysis of the Hess Corporation analyst report, categorized as requested:

**Reasoning:**

* **Finance:** The report highlights Hess's strong cash position, improving balance sheet, and lack of major debt maturities until 2023. It also mentions the company's dividend payments since 1922. These factors point to a positive sentiment regarding Hess's financial health.
* **Production:** The report discusses Hess's production plans and forecasts, including the expected increase in production from the Liza prospect in Guyana and the Bakken Shale. While there's a mention of a potential decline in Bakken production in 2021, the overall focus on production growth suggests a positive outlook.
* **Reserves / Exploration / Acquisitions / Mergers / Divestments:** The report mentions Hess's significant stake in the Stabroek block offshore Guyana, which is a major growth opportunity.  The report also mentions asset sales, but these are presented as strategic moves to sharpen focus

In [12]:
# Get another random analyst report
company, source, headline, text, id, url = get_random_text(text_df, 'Investment Research', random_state=41)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: IR-1309

Company: DVN

Source: Investment Research

Headline: Devon Energy Corporation

URL: nan

Text:
Stock Report | December 19, 2020 | NYSE Symbol: DVN | DVN is in the S&P 500
Devon Energy Corporation
Recommendation BUY « « « « « Price 12-Mo. Target Price Report Currency Investment Style
USD 15.63 (as of Dec 18, 2020) USD 12.00 USD Mid-Cap Value
Equity Analyst Stewart Glickman, CFA
GICS Sector Energy Summary Devon Energy is one of the largest independent oil & gas exploration & production companies
Sub-Industry Oil and Gas Exploration and Production in the U.S.
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 26.54 - 5.41 Oper.EPS2020E USD -0.24 Market Capitalization[B] USD 5.98 Beta 3.29
Trailing 12-Month EPS USD -6.56 Oper.EPS2021E USD 0.52 Yield [%] 2.82 3-yr Proj. EPS CAGR[%] -9
Trailing 12-Month P/E NM P/E on Oper.EPS2020E NM Dividend Rate/Share USD 0.44 SPGMI's Quality Ranking B-
USD 10K Invested 5 Yrs Ago

In [13]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's the sentiment analysis of the analyst report for DVN, along with reasoning and references from the text:

**Finance** 

- **Sentiment: Positive** 

- **Reasoning:** The report highlights DVN's positive financial trends, including a decrease in long-term debt, an increase in cash on hand, and plans to reduce debt relative to EBITDA. The report also notes that DVN's return on invested capital is slightly above the peer median.  
- **References:**  "Its long-term debt at year-end 2019 stood at $4.3 billion, down dramatically from $10.3 billion at year end 2017, and we think it is headed in the right direction."  "The pending sale of DVN’s Barnett Shale assets on October 1, 2020 for up to $830 million (cash proceeds of $570 million and contingent payments of up to $260 million) should help buttress DVN’s war chest, which now stands at $1.5 billion cash on hand at June 30, 2020." "We estimate return on invested capital in 2020 of 3.5%, slightly above the 2.7% peer median."

**Product

# Test SEC Filings

In [14]:
# Get a random SEC Filing
company, source, headline, text, id, url = get_random_text(text_df, 'SEC Filings', random_state=1)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: SEC-82118924000002

Company: EOG

Source: SEC Filings

Headline: 8-K

URL: https://www.sec.gov/Archives/edgar/data/0000821189/000082118924000002/eog-20240111.htm

Text:
0000821189false00008211892024-01-112024-01-11UNITED STATESSECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549FORM 8-K CURRENT REPORTPursuant to Section 13 OR 15(d) of the Securities Exchange Act of 1934Date of Report (Date of earliest event reported):  January 11, 2024_______________EOG RESOURCES, INC. (Exact name of registrant as specified in its charter)1111 Bagby, Sky Lobby 2 Houston, Texas  77002  (Address of principal executive offices) (Zip Code)713-651-7000  (Registrant's telephone number, including area code)Check the appropriate box below if the Form 8-K filing is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions:☐    Written communications pursuant to Rule 425 under the Securities Act (17 CFR 230.425)☐    Soliciting material pursua

In [15]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here is the sentiment analysis of the provided SEC filing text:

**Reasoning:**

* **Finance:** The text discusses EOG's financial commodity derivative contracts, highlighting an anticipated net gain of $298 million and net cash received from settlements of $18 million. This indicates positive financial performance related to these contracts. 
* **Production:** While production levels are not explicitly discussed, the mention of crude oil and natural gas realizations and adjustments suggests ongoing production activities. The document lacks information indicating positive or negative sentiment towards production.
* **Reserves / Exploration / Acquisitions / Mergers / Divestments:**  The text mentions EOG's efforts to acquire and discover additional reserves and its plans for developing acreage and maximizing reserve recovery. This signals a positive sentiment towards exploration and development.
* **Environment / Regulatory / Geopolitics:**  The text explicitly addresses the impact of c

In [16]:
# Get another random SEC Filing
company, source, headline, text, id, url = get_random_text(text_df, 'SEC Filings', random_state=6)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: SEC-9341022000026

Company: CVX

Source: SEC Filings

Headline: 10-Q

URL: https://www.sec.gov/Archives/edgar/data/0000093410/000009341022000026/cvx-20220331.htm

Text:
000009341012/312022Q1falseRefer to Note 12 Other Contingencies and Commitments.00000934102022-01-012022-03-3100000934102022-03-31xbrli:sharesiso4217:USD00000934102021-01-012021-03-31iso4217:USDxbrli:shares00000934102021-12-3100000934102020-12-3100000934102021-03-310000093410us-gaap:CommonStockIncludingAdditionalPaidInCapitalMember2020-12-310000093410us-gaap:RetainedEarningsMember2020-12-310000093410us-gaap:AociIncludingPortionAttributableToNoncontrollingInterestMember2020-12-310000093410us-gaap:TreasuryStockMember2020-12-310000093410us-gaap:ParentMember2020-12-310000093410us-gaap:NoncontrollingInterestMember2020-12-310000093410us-gaap:CommonStockIncludingAdditionalPaidInCapitalMember2021-01-012021-03-310000093410us-gaap:ParentMember2021-01-012021-03-310000093410us-gaap:RetainedEarningsMember2021-01-012021-03-

In [17]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here is the sentiment analysis of the provided text from the SEC filing, categorized as requested:

**Reasoning:**

This SEC filing primarily focuses on Chevron's financial performance and its outlook for the future.  The document  highlights positive financial results, particularly strong earnings due to higher oil prices, but also acknowledges concerns and uncertainties related to various factors, including the war in Ukraine, global economic conditions, and climate change regulations. 

**Category Breakdown:**

* **Finance - Positive:** The text emphasizes Chevron's strong financial performance with record earnings driven by high oil prices. Mentions of share repurchases, dividend payments, and commitment to high-quality debt ratings suggest financial strength and shareholder-friendly actions.
* **Production - Neutral:**  While production numbers are mentioned, there's no significant positive or negative sentiment expressed about production levels or trends. The text focuses more on

# Test Earnings Call Transcripts

In [18]:
# Get an earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Presentations', random_state=42)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EP-210

Company: SHEL

Source: Earnings Call Presentations

Headline: Royal Dutch Shell plc, Q3 2021 Earnings Call, Oct 28, 2021

URL: nan

Text:
Royal Dutch Shell plc ENXTAM:RDSA
FQ3 2021 Earnings Call Transcripts
Thursday, October 28, 2021 1:00 PM GMT
S&P Global Market Intelligence Estimates
-FQ3 2021- -FQ4 2021- -FY 2021- -FY 2022-
CONSENSUS ACTUAL SURPRISE CONSENSUS CONSENSUS CONSENSUS
EPS Normalized 0.70 0.53 (24.29 %) 0.92 2.45 3.02
Revenue (mm) 63399.50 60044.00 (5.29 %) 63362.65 301074.06 299662.46
Currency: USD
Consensus as of Oct-28-2021 8:36 AM GMT
- EPS NORMALIZED -
CONSENSUS ACTUAL SURPRISE
FQ4 2020 0.08 0.05 (37.50 %)
FQ1 2021 0.43 0.42 (2.33 %)
FQ2 2021 0.62 0.71 12.70 %
FQ3 2021 0.70 0.53 (24.29 %)
1
COPYRIGHT © 2021 S&P Global Market Intelligence, a division of S&P Global Inc. All rights reserved
spglobal.com/marketintelligence

ROYAL DUTCH SHELL PLC FQ3 2021 EARNINGS CALL | OCT 28, 2021
Presentation
Operator
Welcome to the Royal Shell 2021 Q3 Results Announ

In [19]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of Shell's Q3 2021 Earnings Call

**Reasoning:** 

The provided text highlights Shell's strong financial performance, particularly its cash generation and progress towards net-zero emissions. This positive outlook on financial strength and sustainability efforts suggests a positive sentiment. 

The mention of "economic recovery" and "well positioned" implies optimism about the future. However, the brief discussion of Third Point's letter, while stating Shell's commitment to shareholder engagement, may introduce a degree of uncertainty, potentially leaning towards a neutral sentiment in finance. 

The text focuses on Shell's overall strategy and performance rather than specific production details, leading to a neutral sentiment in the production category. 

There is no mention of reserves, exploration, acquisitions, mergers, or divestments, making this category neutral. 

The emphasis on becoming a "net-zero emissions energy business" indicates a positive sentiment

In [20]:
# Get another earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Presentations', random_state=41)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EP-43

Company: HES

Source: Earnings Call Presentations

Headline: Hess Corporation, Q2 2019 Earnings Call, Jul 31, 2019

URL: nan

Text:
Hess Corporation NYSE:HES
FQ2 2019 Earnings Call Transcripts
Wednesday, July 31, 2019 2:00 PM GMT
S&P Global Market Intelligence Estimates
-FQ2 2019- -FQ3 2019- -FY 2019- -FY 2020-
CONSENSUS ACTUAL SURPRISE CONSENSUS CONSENSUS CONSENSUS
EPS Normalized (0.12) (0.09) NM (0.16) (0.16) 1.25
Revenue (mm) 1518.93 1697.00 11.72 1570.32 6444.65 7650.09
Currency: USD
Consensus as of Jul-31-2019 2:23 PM GMT
- EPS NORMALIZED -
CONSENSUS ACTUAL SURPRISE
FQ3 2018 0.02 0.38 1800.00 %
FQ4 2018 (0.40) (0.31) NM
FQ1 2019 (0.26) 0.09 NM
FQ2 2019 (0.12) (0.09) NM
1
COPYRIGHT © 2019 S&P Global Market Intelligence, a division of S&P Global Inc. All rights reserved
spglobal.com/marketintelligence

HESS CORPORATION FQ2 2019 EARNINGS CALL | JUL 31, 2019
Presentation
Operator
Good day, ladies and gentlemen, and welcome to the Second Quarter 2019 Hess Corporation 

In [21]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's a breakdown of the sentiment analysis for each category based on the Hess Corporation Q2 2019 Earnings Call transcript:

**Finance:** 

- **Sentiment: Positive** 
- **Reasoning:**  The company highlights strong operational performance, increased production, and expectations of industry-leading cash flow growth. They mention a decrease in breakeven costs and a significant increase in free cash flow generation. This points to a positive financial outlook.

**Production:**

- **Sentiment: Positive**
- **Reasoning:**  The call emphasizes exceeding production expectations, with full-year production projected at the upper end of the guidance range. The Bakken is highlighted as a key driver of this growth, along with successful ramp-up efforts. 

**Reserves / Exploration / Acquisitions / Mergers / Divestments:**

- **Sentiment: Positive**
- **Reasoning:** The call focuses on the expansion of resources in Guyana, with the Stabroek Block showing significant potential. The discovery of ne

# Test Earnings Call Q&A Sessions

In [23]:
# Get an earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Q&A', random_state=41)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EQ-130

Company: DVN

Source: Earnings Call Q&A

Headline: Devon Energy Corporation, Q4 2021 Earnings Call, Feb 16, 2022

URL: nan

Text:
Question and Answer
Operator
[Operator Instructions] Our first question is from Arun Jayaram with JPMorgan.
Arun Jayaram
JPMorgan Chase & Co, Research Division
Perhaps for Jeff, I wanted to get more insights on the base dividend increase and how we should think
about the mix of base and variable dividends in future periods? Because I believe the -- historically, you
guys have targeted about 10% of CFO at, call it, a mid-cycle price for the base dividend. So I want to get
some thoughts on how that's evolving over time.
Jeffrey L. Ritenour
Executive VP & CFO
Yes, you bet, Arun. Thanks for the question. You're spot on. Historically, the way we thought about that is
kind of on a normalizing -- normalized price [ deck ], kind of a mid-cycle price deck. We've targeted kind
of a 5% to 10% payout ratio of our cash flow for that fixed dividend. So 

In [24]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of DVN Earnings Call Q&A

The Q&A session focuses primarily on Devon Energy's financial performance, production strategy, and capital allocation plans in light of the current oil and gas market environment. The sentiment across various categories is as follows:

- Finance - Positive: The company highlights its strong financial position, with increasing fixed dividends and continued share repurchases.
- Production - Neutral: Devon plans to maintain production at a steady state level, with growth focused in the Permian Basin. The company acknowledges the potential for growth, but stresses its commitment to disciplined capital allocation.
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral:  There is no mention of significant exploration plans, acquisitions, mergers, or divestments. 
- Environment / Regulatory / Geopolitics - Neutral: The Q&A does not address environmental concerns, regulatory pressures, or geopolitical risks.
- Alternative Ener

In [25]:
# Get another earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Q&A', random_state=11)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EQ-116

Company: MRO

Source: Earnings Call Q&A

Headline: Marathon Oil Corporation, Q2 2021 Earnings Call, Aug 05, 2021

URL: nan

Text:
Question and Answer
Operator
[Operator Instructions]
And I see we have our first question from Scott Hanold with RBC Capital Markets.
Scott Michael Hanold
RBC Capital Markets, Research Division
I think the updated shareholder return plan is pretty interesting and exciting, frankly, if I'm an investor.
And Dane, I think you had made mention of, look, it's something you all don't have to necessarily wait
on until 2022. Can you give us a sense of like how do you think about timing, is this more likely like,
look, once you guys get that debt reduction done in September, that's when you'd look at it. And just
kind of curious on the go-forward plan. Is this something you evaluate once a quarter is done, what the
prices were, what the cash flow is to make that determination? Just -- is there any kind of structure
of just getting a sense of how we

In [26]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

The earnings call transcript focuses heavily on Marathon Oil's financial performance and shareholder return strategy, with less emphasis on production, reserves, or environmental concerns. Here's a breakdown of the sentiment in each category:

- Finance - Positive 
    - The company expresses confidence in its financial performance, highlighting its commitment to returning capital to shareholders, debt reduction, and achieving a strong balance sheet. 
- Production - Neutral
    - The discussion about production primarily revolves around maintaining current levels and the wells to sales profile, with no significant positive or negative sentiments expressed.
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
    - The call mentions resource capture opportunities, but the focus is on organic enhancement within existing assets rather than large-scale acquisitions, mergers, or divestments. 
- Environment / Regulatory / Geopolitics - Neutral 
    - No specific mention 